## **Mapas do Espírito Santo**

### **Carregando o Shapefile**

In [3]:
# Instalando a biblioteca geopandas
%pip install geopandas --quiet

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Instalando a biblioteca folium
%pip install folium --quiet

Note: you may need to restart the kernel to use updated packages.


In [6]:
# Importando as bibliotecas necessárias
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import MultiPolygon
import matplotlib.pyplot as plt
import folium